<a href="https://colab.research.google.com/github/amitkp57/Jupyter/blob/main/Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Move to GPU mode if you are in Google Colab
Go to `Runtime` -> `Change runtime type` to activate GPU.

### Provide your credentials to the runtime


In [10]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Mount Google Drive

In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
WORKING_DIRECTORY = '/content/gdrive/MyDrive/Data/Jupyter'
os.environ['WORKING_DIRECTORY'] = WORKING_DIRECTORY
%cd $WORKING_DIRECTORY
!ls -latr

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Data/Jupyter
total 25
-rw------- 1 root root  530 Mar 10 12:53 requirements.txt
-rw------- 1 root root   85 Mar 10 12:53 main.py
-rw------- 1 root root 1069 Mar 10 12:53 LICENSE
-rw------- 1 root root 3565 Mar 10 12:53 Jupyter.ipynb
-rw------- 1 root root 1824 Mar 10 12:53 .gitignore
drwx------ 2 root root 4096 Mar 10 12:53 data
-rw------- 1 root root    9 Mar 10 15:41 README.md
drwx------ 3 root root 4096 Mar 10 15:41 scripts
drwx------ 8 root root 4096 Mar 10 15:41 .git
-rw------- 1 root root    0 Mar 10 15:41 app.log
drwx------ 3 root root 4096 Mar 10 16:17 results


### Clone git repo

In [ ]:
%cd $WORKING_DIRECTORY
!git config --global user.email "amitpradhanorissa@gmail.com"
!git config --global user.name "Amit Pradhan"
# !git clone https://github.com/amitkp57/Jupyter
!git reset --hard
!git pull origin main
# !pip install -r requirements.txt
!pip install datasketch

### Setup meta data

Creates datasets.txt, tables.txt and columns.json in the /data folder. These files are used for querying Google Bigquery tables. 

In [14]:
from scripts.MetaData import save_locally
DATA_PATH = f'{WORKING_DIRECTORY}/data'
os.environ[
        'GOOGLE_APPLICATION_CREDENTIALS'] = f'{DATA_PATH}/amit-pradhan-compute-23315413b3a3.json'
# save_locally(DATA_PATH)
print('Completed!')

Completed!


### Jaccard similarity

Jaccard similarity between columns based on minHash

In [15]:
import scripts.QueryDatabase as queryDatabase
import scripts.ClusterColumns as clusterColumns
import numpy as np

RESULTS_PATH = f'{WORKING_DIRECTORY}/results'
string_columns = queryDatabase.get_columns('STRING')
# np.savez(f'{RESULTS_PATH}/string_columns.npz', string_columns=string_columns)
clusterColumns.serialize_min_hash(string_columns)
similarity = clusterColumns.calculate_jaccard_similarity(string_columns)
print(np.unique(np.round(similarity.ravel(), 2)))
print('Completed!')

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27
 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41
 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55
 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69
 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83
 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97
 0.98 0.99 1.  ]
Completed!


### Find columns with Jaccard similarity greater than a threshold value

We use MinHash based LSH t find columns with Jaccard similarity greater than a given threshold value.

In [16]:
import scripts.QueryDatabase as queryDatabase
import scripts.ClusterColumns as clusterColumns

RESULTS_PATH = f'{WORKING_DIRECTORY}/results'
string_columns = queryDatabase.get_columns('STRING')
# np.savez(f'{RESULTS_PATH}/string_columns.npz', string_columns=string_columns)
lsh = clusterColumns.build_minhash_lsh(string_columns, threshold=0.7)
print(clusterColumns.get_all_similar_columns(lsh, string_columns[0]))
print('Completed!')

['bigquery-public-data.covid19_nyt.us_counties.county_fips_code', 'bigquery-public-data.covid19_jhu_csse.summary.fips', 'bigquery-public-data.covid19_usafacts.deaths.county_fips_code', 'bigquery-public-data.covid19_aha.hospital_beds.county_fips_code', 'bigquery-public-data.covid19_public_forecasts_asia_ne1.county_28d.county_fips_code', 'bigquery-public-data.covid19_usafacts.confirmed_cases.county_fips_code', 'bigquery-public-data.covid19_weathersource_com.county_day_history.county_fips_code', 'bigquery-public-data.covid19_public_forecasts.county_14d.county_fips_code', 'bigquery-public-data.covid19_public_forecasts.county_14d_historical_.county_fips_code', 'bigquery-public-data.covid19_symptom_search.symptom_search_sub_region_2_daily.sub_region_2_code', 'bigquery-public-data.covid19_symptom_search.symptom_search_sub_region_2_weekly.sub_region_2_code', 'bigquery-public-data.covid19_public_forecasts.county_14d_historical.county_fips_code', 'bigquery-public-data.covid19_google_mobility.mob

### Top-k similar columns based on Jaccard similarity

We use minhash LSH based forest to query top k columns based on Jaccard similarity

In [17]:
import scripts.QueryDatabase as queryDatabase
import scripts.ClusterColumns as clusterColumns

string_columns = queryDatabase.get_columns('STRING')
forest = clusterColumns.build_lsh_forest(string_columns)
print(clusterColumns.get_top_k(forest, string_columns[0], 10))
print('Completed!')

['bigquery-public-data.covid19_aha.staffing.county_fips_code', 'bigquery-public-data.covid19_google_mobility.mobility_report.census_fips_code', 'bigquery-public-data.covid19_google_mobility_eu.mobility_report.census_fips_code', 'bigquery-public-data.covid19_nyt.us_counties.county_fips_code', 'bigquery-public-data.covid19_public_forecasts.county_14d.county_fips_code', 'bigquery-public-data.covid19_jhu_csse.summary.fips', 'bigquery-public-data.covid19_jhu_csse_eu.summary.fips', 'bigquery-public-data.covid19_symptom_search.symptom_search_sub_region_2_daily.sub_region_2_code', 'bigquery-public-data.covid19_aha.hospital_beds.county_fips_code', 'bigquery-public-data.covid19_nyt.mask_use_by_county.county_fips_code']
Completed!


### ScratchPad
Try anything here!

In [8]:
import scripts.QueryDatabase as queryDatabase
import scripts.ClusterColumns as clusterColumns
import numpy as np

RESULTS_PATH = f'{WORKING_DIRECTORY}/results'
string_columns = queryDatabase.get_columns('STRING')
# np.savez(f'{RESULTS_PATH}/string_columns.npz', string_columns=string_columns)
clusterColumns.serialize_min_hash(string_columns)
similarity = clusterColumns.calculate_jaccard_similarity(string_columns)
# print(np.unique(np.round(similarity.ravel(), 2)))
print('Completed!')

Completed!
